In [2]:
import pandas as pd
import random

# Veri setini yükleyelim
df = pd.read_csv('supermarket_sales.csv')

# Benzersiz ürünleri alalım
unique_products = df['Product'].unique()

# Ürünleri karıştıralım
random.shuffle(unique_products)

# 168 ürünü 2'şerli gruplara ayıralım
product_pairs = [(unique_products[i], unique_products[i+1]) for i in range(0, len(unique_products), 2)]

# Toplamda 84 grup olmasını sağlayalım
comb_df = pd.DataFrame(product_pairs, columns=['Product_1', 'Product_2'])

# İlk 5 satırı kontrol edelim
print(comb_df.head())


               Product_1                       Product_2
0       - Tuvalet kağıdı               - Hijyen ürünleri
1       - Yumuşak peynir           - Bulaşık temizleyici
2  - Evcil hayvan bakımı                   - Kağıt havlu
3        - Çöp torbaları  - Organik dondurulmuş sebzeler
4      - Şişelenmiş bira                    - Rulo/çörek


In [3]:
comb_df

,Product_1,Product_2
0,- Tuvalet kağıdı,- Hijyen ürünleri
1,- Yumuşak peynir,- Bulaşık temizleyici
2,- Evcil hayvan bakımı,- Kağıt havlu
3,- Çöp torbaları,- Organik dondurulmuş sebzeler
4,- Şişelenmiş bira,- Rulo/çörek
...,...,...
57,- Dezenfektan,- Sirke
58,- Hazır kahve,- Köpüklü şarap
59,- Şekillendirici jel,- Tatlı
60,- Meyve/sebze suyu,- Özel rulo/çörek


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 1. Adım: Veri Yükleme
df = pd.read_csv('supermarket_sales.csv')

# 2. Adım: Ürün Gruplarını ve Rating Değerlerini Eşleştirme
unique_products = df['Product'].unique()
random.shuffle(unique_products)
product_pairs = [(unique_products[i], unique_products[i + 1]) for i in range(0, len(unique_products), 2)]

# Ürün çifti ve rating farkı ile bir DataFrame oluştur
data = []
for p1, p2 in product_pairs:
    rating_p1 = df[df['Product'] == p1]['Rating'].mean()
    rating_p2 = df[df['Product'] == p2]['Rating'].mean()
    
    # NaN Kontrolü
    if pd.isna(rating_p1) or pd.isna(rating_p2):
        continue  # Eğer herhangi bir rating NaN ise, bu satırı atla

    rating_diff = abs(rating_p1 - rating_p2)
    # Hedef değişken oluştur
    label = 1 if rating_diff <= 1.1 else 0
    data.append({'Product_1': p1, 'Product_2': p2, 'Rating_Diff': rating_diff, 'Label': label})

pairs_df = pd.DataFrame(data)

# 3. Adım: Model Eğitimi
X = pairs_df[['Rating_Diff']]
y = pairs_df['Label']

# NaN değerleri kaldırma (eksik satırlar)
X = X.dropna()
y = y[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

# 5. Adım: Tahmin Fonksiyonu
def recommend_placement(product_pairs, model):
    recommendations = []
    for p1, p2 in product_pairs:
        rating_diff = abs(df[df['Product'] == p1]['Rating'].mean() - df[df['Product'] == p2]['Rating'].mean())
        
        # NaN kontrolü
        if pd.isna(rating_diff):
            continue  # Eğer rating farkı NaN ise, bu satırı atla
        
        prediction = model.predict([[rating_diff]])
        if prediction == 1:
            recommendations.append(f"{p1} ve {p2} birlikte yerleştirilebilir.")
        else:
            recommendations.append(f"{p1} ve {p2} uzakta yerleştirilmelidir.")
    return recommendations

# Tüm ürün çiftleri için tahmin
results = recommend_placement(product_pairs, model)
for result in results:
    print(result)


- Bebek kozmetiği ve - Hazır kahve uzakta yerleştirilmelidir.
- Yoğurt ve - Meyve/sebze suyu uzakta yerleştirilmelidir.
- İşlenmiş peynir ve - Hardal birlikte yerleştirilebilir.
- Beyaz şarap ve - Organik sosis birlikte yerleştirilebilir.
- Organik konserve sebzeler ve - Bulaşık temizleyici uzakta yerleştirilmelidir.
- Gazlı içecek ve - Şurup uzakta yerleştirilmelidir.
- Dondurulmuş sebzeler ve - Krem şanti uzakta yerleştirilmelidir.
- Lor ve - Yumuşak peynir uzakta yerleştirilmelidir.
- Organik baharatlar ve - Çöp torbaları uzakta yerleştirilmelidir.
- Hamur işi ve - Şişelenmiş su uzakta yerleştirilmelidir.
- Konserve bira ve - Özel çikolata uzakta yerleştirilmelidir.
- Meyveler ve - Margarin uzakta yerleştirilmelidir.
- Fırın ürünleri ve - Organik meyve suyu uzakta yerleştirilmelidir.
-Rakı ve - Evcil hayvan bakımı uzakta yerleştirilmelidir.
- Makyaj temizleyici ve - Uzun ömürlü fırın ürünü uzakta yerleştirilmelidir.
- Meyve suyu ve - Yerli yumurta uzakta yerleştirilmelidir.
- Şekerl

C:\Users\90553\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\90553\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\90553\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\90553\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\90553\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\90553\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X d

In [9]:
pip install flask


Note: you may need to restart the kernel to use updated packages.
